Dans un 1er temps il faut mettre le fichier sous le meme format que les fichiers camera.


In [6]:
import pandas as pd

df = pd.read_excel('./P2/P2_semaine_2.xlsx')
df['Horodatage'] = pd.to_datetime(df['Horodatage'])
df['Horodatage'] = df['Horodatage'].dt.floor('H')
df['Sens'] = df['Voie'].replace({'Entrée Fac': 0, 'Sortie Fac': 1})
df.drop('Voie', axis=1, inplace=True)
df['Count'] = 1

pivot_table = df.pivot_table(index=['Horodatage', 'Sens'], columns='Classe', values='Count', aggfunc='sum', fill_value=0)
pivot_table.columns = [col for col in pivot_table.columns] 
pivot_table.reset_index(inplace=True)


pivot_table.to_excel('./P2_2.xlsx', index=False)


C:\Users\Admin\AppData\Local\Temp\ipykernel_11708\2500344291.py:9: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df['Horodatage'] = df['Horodatage'].dt.floor('H')
C:\Users\Admin\AppData\Local\Temp\ipykernel_11708\2500344291.py:11: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['Sens'] = df['Voie'].replace({'Entrée Fac': 0, 'Sortie Fac': 1})


Maintenat faut assembler les 2 valeurs et les mettre dans un format similaire aux capteurs camera meme si y a un manque de type.

Datetime,DayOfWeek,Sens,VELO,MOTO,VL,UT,EDPM,PL_1,PL_2,BUS

In [1]:
import pandas as pd


df1 = pd.read_excel('./P2_1.xlsx')
df2 = pd.read_excel('./P2_2.xlsx')

df_concatenated = pd.concat([df1, df2], ignore_index=True)

df_concatenated.rename(columns={'Horodatage': 'Datetime'}, inplace=True)


df_concatenated['DayOfWeek'] = df_concatenated['Datetime'].dt.dayofweek
columns_order = ['Datetime', 'DayOfWeek', 'Sens', 'Deux roues', 'VL', 'PL', 'Bus']
df_final = df_concatenated[columns_order]

print(df_final.head())

df_final.to_csv('./P2.csv', index=False)


             Datetime  DayOfWeek  Sens  Deux roues  VL  PL  Bus
0 2022-10-03 10:00:00          0     0           1   8   0    3
1 2022-10-03 10:00:00          0     1           0   4   0    2
2 2022-10-03 11:00:00          0     0          16  51   1    8
3 2022-10-03 11:00:00          0     1           5  52   2   10
4 2022-10-03 12:00:00          0     0           9  69   0   11


Alooors, pour P3 on trouve plein de données manquante ste des vides entre le début et la fin des fichiers.

je vais travailler avec mixtra car c'est eux qui possede la majorité des fichiers.

In [7]:
import pandas as pd

df = pd.read_csv('./P3/Mixtra_Sortie_Fac_3.csv')

df['Sens'] = 1
df['Heure_juste'] = df['Heure'].str.split(':').str[0]
df['Datetime'] = pd.to_datetime(df['Jour'] + ' ' + df['Heure_juste'] + ':00:00', format='%d/%m/%y %H:%M:%S')
df['Count'] = 1

df['Datetime'] = df['Datetime'].dt.floor('H')
df.drop(['Jour', 'Heure', 'Seconde', 'Centième','Vitesse','Catégorie','Inter-essieux','E2/E3','E3/E4','E4/E5','E5/E6'], axis=1, inplace=True)

pivot_table = df.pivot_table(index=['Datetime', 'Sens'], columns='Type Véhicule', values='Count', aggfunc='count', fill_value=0)

pivot_table.columns = [f"{col}" for col in pivot_table.columns]
pivot_table.reset_index(inplace=True)


pivot_table.to_excel('./P3/Mixtra_Sortie_Fac_3.xlsx', index=False)


C:\Users\Admin\AppData\Local\Temp\ipykernel_5388\770440663.py:13: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df['Datetime'] = df['Datetime'].dt.floor('H')


Ensuite faudra fusionner les fichiers et remplir avec les plages horaires manquantes, afin de les completer ces valeurs manquantes.

In [8]:
import pandas as pd
import numpy as np

df1 = pd.read_excel('./P3/Mixtra_Sortie_Fac_1.xlsx')
df2 = pd.read_excel('./P3/Mixtra_Sortie_Fac_2.xlsx')
df3 = pd.read_excel('./P3/Mixtra_Sortie_Fac_3.xlsx')

df_concatenated = pd.concat([df1, df2,df3], ignore_index=True)

df_concatenated['DayOfWeek'] = df_concatenated['Datetime'].dt.dayofweek

# Réorganiser les colonnes
cols_order = ['Datetime', 'DayOfWeek', 'Sens', 'Trottinette', 'Vélo', 'Moto', 'VL', 'PL']
df_concatenated = df_concatenated[cols_order]


df_concatenated.to_csv('p3_S.csv', index=False)


In [11]:
import pandas as pd

df1 = pd.read_csv('./P3_E.csv')
df2 = pd.read_csv('./P3_S.csv')

df_concatenated = pd.concat([df1, df2], ignore_index=True)
df_concaten = df_concatenated.sort_values(by='Datetime')
df_concaten.to_csv('p3.csv', index=False)


Pour le fichier P4.

In [28]:
import pandas as pd

df_entree = pd.read_csv('./P4/P4_Sortie_Fac.csv')

base_date = '2022-10-'
df_entree['Datetime'] = pd.to_datetime(
    base_date + df_entree['JOUR'].astype(str).str.zfill(2) + ' ' +
    df_entree['HEURE/MINUTE'].astype(str).str.zfill(4).str.slice(0, 2) + ':00:00'
)
df_entree['Count'] = 1
df_entree['Datetime'] = df_entree['Datetime'].dt.floor('H')
df_entree['DayOfWeek'] = df_entree['Datetime'].dt.dayofweek
df_entree['Sens'] = 1

pivot_table = df_entree.pivot_table(index=['Datetime', 'DayOfWeek', 'Sens'], columns='TYPE', values='Count', aggfunc='count', fill_value=0)
pivot_table.reset_index(inplace=True)
cols_order = ['Datetime', 'DayOfWeek', 'Sens', '2R', 'VL', 'PL/Bus']
pivot_table = pivot_table[cols_order] 
pivot_table.to_csv('P4_S.csv', index=False)


C:\Users\Admin\AppData\Local\Temp\ipykernel_5388\1520496975.py:11: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df_entree['Datetime'] = df_entree['Datetime'].dt.floor('H')


Rassembler le code entree et sortie en supprimant la colonne vélo.

In [26]:
import pandas as pd

df = pd.read_csv('./P5.csv')

unique_types = df['TYPE'].unique()
print("Types présents dans les données :", unique_types)


Types présents dans les données : ['VL' 'PL' '2R']


In [21]:
import pandas as pd

df1 = pd.read_csv('./P4_E.csv')
df2 = pd.read_csv('./P4_S.csv')

df1['2R'] += df1['Vélo']
df1.drop('Vélo', axis=1, inplace=True)

df_concatenated = pd.concat([df1, df2], ignore_index=True)
df_concaten = df_concatenated.sort_values(by='Datetime')
df_concaten.to_csv('P4.csv', index=False)
